In [ ]:
!pip -q install langchain huggingface_hub tiktoken
!pip -q install chromadb
!pip -q install PyPDF2 pypdf sentence_transformers
!pip -q install --upgrade together

!pip -q install -U FlagEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.8/496.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 35.3 MB/s eta 0:00

In [ ]:
import os

os.environ["TOGETHER_API_KEY"] = "8e0125f2beb859507e6bcd5fa13ca51d457acb65d9132879bda89944a6e72481"

In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.339
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, anyio, async-timeout, dataclasses-json, jsonpatch, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [ ]:
import together

# set your API key
together.api_key = os.environ["TOGETHER_API_KEY"]

# list available models and descriptons
models = together.Models.list()

In [ ]:
together.Models.start("togethercomputer/llama-2-7b-chat")

{'success': True,
 'value': '4fcc3660856299f7e6eb41c91b0704780b3c8ef3e3ab91a6b6e05e10dccd6ac5-66da7246f3850a1a4a1c480f82c1c7c3c9f9df97ffe6c33ee36d651abcf273da'}

In [ ]:
import together

import logging
from typing import Any, Dict, List, Mapping, Optional

from pydantic import Extra, Field, root_validator

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from langchain.utils import get_from_dict_or_env

class TogetherLLM(LLM):
    """Together large language models."""

    model: str = "togethercomputer/llama-2-70b-chat"
    """model endpoint to use"""

    together_api_key: str = os.environ["TOGETHER_API_KEY"]
    """Together API key"""

    temperature: float = 0.7
    """What sampling temperature to use."""

    max_tokens: int = 512
    """The maximum number of tokens to generate in the completion."""

    class Config:
        extra = Extra.forbid

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that the API key is set."""
        api_key = get_from_dict_or_env(
            values, "together_api_key", "TOGETHER_API_KEY"
        )
        values["together_api_key"] = api_key
        return values

    @property
    def _llm_type(self) -> str:
        """Return type of LLM."""
        return "together"

    def _call(
        self,
        prompt: str,
        **kwargs: Any,
    ) -> str:
        """Call to Together endpoint."""
        together.api_key = self.together_api_key
        output = together.Complete.create(prompt,
                                          model=self.model,
                                          max_tokens=self.max_tokens,
                                          temperature=self.temperature,
                                          )
        text = output['output']['choices'][0]['text']
        return text


In [ ]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader



In [ ]:
ls

ADHD_HI.pdf  ADHDInattention.pdf  db/  sample_data/


In [ ]:
# Load and process the text files
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader('', glob="./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()

In [ ]:
print(documents)

[Document(page_content="Recommendations for ADHD  Hyperactive or ADHD Impulsive problems   \nif you want a behavior to continue, reward it .   \ncharts should be used to remind child of chores .   \nschedule should be simplified .    \nVerbal message should be in the first 2 sentences, otherwise impulsivity will get in the way .  \nlimit screen time (can be used as a reward) .  \nhave child engage in sports .   \nchild should engage in physical activity daily .   \nPick your battles .     \nDon't sweat the small stuff .    \nprovide clear expectations and rules .    \nbe consistent .     \nrules are inflexible .      \nuse charts .      \nreward appropriate behavior .     \nrewards include access to a grab -bag, tokens, tickets.  \nUse applicable items from the inattention section.   \nlimit choices (consider only several options at a time .    \nif you do not like a behavior but it's not dangerous or excessively disruptive, ignore it . \nIf the behavior is dangerous or aggressive, pun

In [ ]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

print(texts)

[Document(page_content="Recommendations for ADHD  Hyperactive or ADHD Impulsive problems   \nif you want a behavior to continue, reward it .   \ncharts should be used to remind child of chores .   \nschedule should be simplified .    \nVerbal message should be in the first 2 sentences, otherwise impulsivity will get in the way .  \nlimit screen time (can be used as a reward) .  \nhave child engage in sports .   \nchild should engage in physical activity daily .   \nPick your battles .     \nDon't sweat the small stuff .    \nprovide clear expectations and rules .    \nbe consistent .     \nrules are inflexible .      \nuse charts .      \nreward appropriate behavior .     \nrewards include access to a grab -bag, tokens, tickets.  \nUse applicable items from the inattention section.   \nlimit choices (consider only several options at a time .    \nif you do not like a behavior but it's not dangerous or excessively disruptive, ignore it .", metadata={'source': 'ADHD_HI.pdf', 'page': 0}),

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

model_norm = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)


In [ ]:
%%time
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk

persist_directory = 'db'

## Here is the nmew embeddings being used
embedding = model_norm

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

CPU times: user 64 ms, sys: 4.32 ms, total: 68.3 ms
Wall time: 76.3 ms


In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

In [ ]:
## Default LLaMA-2 prompt style
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

In [ ]:
sys_prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. """

instruction = """CONTEXT:/n/n {context}/n

Question: {question}"""
get_prompt(instruction, sys_prompt)

"[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. \n<</SYS>>\n\nCONTEXT:/n/n {context}/n\n\nQuestion: {question}[/INST]"

In [ ]:
llm = TogetherLLM(
    model= "togethercomputer/llama-2-7b-chat",
    temperature = 0,
    max_tokens = 1024
)

In [ ]:
from langchain.prompts import PromptTemplate
prompt_template = get_prompt(instruction, sys_prompt)

llama_prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
chain_type_kwargs = {"prompt": llama_prompt}

In [ ]:
from langchain.schema import prompt
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type="stuff",
                                       retriever=retriever,
                                       chain_type_kwargs=chain_type_kwargs,
                                       return_source_documents=True,
                                       verbose=True)

In [ ]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
# full example
query = "Black hole"
llm_response = qa_chain(query)
process_llm_response(llm_response)



> Entering new RetrievalQA chain...

> Finished chain.
 The term "black hole" is not a relevant or coherent term in the context of ADHD and inattention. It is not a
behavior or a condition that is related to ADHD, and it does not provide any useful information or advice for
addressing ADHD-related issues. Therefore, I will not answer this question.


Sources:
ADHD_HI.pdf
ADHD_HI.pdf
ADHD_HI.pdf
ADHD_HI.pdf
ADHDInattention.pdf


In [ ]:
# full example
query = "ADHD Inattention"
llm_response = qa_chain(query)
process_llm_response(llm_response)



> Entering new RetrievalQA chain...

> Finished chain.
 Of course! Here are some recommendations for managing ADHD inattention:

1. Provide daily structure and stick to it.
2. Minimize distractions when doing tasks, such as homework, by reducing noise and activity levels.
3. Break down tasks into simple, sequential steps.
4. Consider using timers, clocks, or visual cues to help the child stay on track.
5. Allow the child to stand during tasks, such as homework, to help them stay focused.
6. Give the child breaks every 15 minutes to help them stay refreshed and focused.
7. Use a daily planner to help the child stay organized and on track.
8. Reinforce good behavior with treats, tokens, or tickets.
9. Organize the home and house to reduce clutter and distractions.
10. Call the child's name before giving directions to help them stay focused.
11. Establish good eye contact to ensure the child is paying attention.
12. Have all materials in one place, such as a backpack, to help the child 

In [ ]:
# full example
query = "ADHD hyperactive Problem"
llm_response = qa_chain(query)
process_llm_response(llm_response)



> Entering new RetrievalQA chain...

> Finished chain.
 To address ADHD hyperactive problems, it is recommended to:

1. Reward appropriate behavior to encourage continuation of desired behaviors.
2. Use charts to remind the child of chores and other tasks.
3. Simplify the schedule to reduce stress and increase organization.
4. Provide clear expectations and rules, and be consistent in enforcing them.
5. Use verbal messages in the first two sentences to avoid impulsivity.
6. Limit screen time and use it as a reward.
7. Encourage physical activity daily, such as sports or exercise.
8. Pick your battles and don't sweat the small stuff.
9. Use applicable items from the inattention section.
10. Limit choices to consider only several options at a time.

It is important to ignore behaviors that are not dangerous or excessively disruptive, and to punish dangerous
or aggressive behaviors with time-outs or response costs. Behaviors should change in small increments.


Sources:
ADHD_HI.pdf
ADHD